In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import math

In [2]:
data=pd.read_csv('data.csv')
data

,datetime,high,low,close,open,volume
0,2018-01-01,13818.55,12750.00,13135.00,13715.65,6970.441076
1,2018-01-02,14750.00,12890.02,14580.00,13135.00,16359.347340
2,2018-01-03,15473.49,14150.00,14749.97,14576.87,19457.238601
3,2018-01-04,15280.00,13918.04,14811.00,14710.00,18783.876561
4,2018-01-05,17118.13,14600.00,16579.00,14805.01,22383.657333
...,...,...,...,...,...,...
1468,2022-01-08,42246.37,40501.00,40679.82,41707.04,27031.900870
1469,2022-01-09,42300.00,40520.00,41966.99,40679.82,27291.009660
1470,2022-01-10,42786.70,39650.00,41604.58,41966.99,51064.904360
1471,2022-01-11,43100.00,41158.99,42755.00,41604.58,38062.818280


In [4]:
output = pd.read_csv('atr_vwap_data.csv')
output

,Unnamed: 0,datetime,signal
0,0,2018-01-01 05:30:00,0
1,1,2018-01-01 06:30:00,0
2,2,2018-01-01 07:30:00,0
3,3,2018-01-01 08:30:00,0
4,4,2018-01-01 09:30:00,0
...,...,...,...
35203,35203,2022-01-12 01:30:00,1
35204,35204,2022-01-12 02:30:00,1
35205,35205,2022-01-12 03:30:00,1
35206,35206,2022-01-12 04:30:00,1


In [5]:
def to_submit(data,output):
    datetime=[]
    trade=[]
    signal=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        if(i==(len(data.close)-1)):
            if(current==1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(-1)
            elif(current==-1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
            else:
                break
            break
        if(output.signal[i]==1):
            if(current==1):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
            elif(current==0):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
                current=1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(1)
                current=0
            
        elif(output.signal[i]==-1):
            if(current==1):
                datetime.append(output.datetime[i])
                signal.append(-1)
#                 price.append(data.close[i])
                trade.append('close')
                current=0
            elif(current==0):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(-1)
                current=-1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
                current=-1
        
        else:
            datetime.append(output.datetime[i])
            trade.append('close')
#             price.append(data.close[i])
            signal.append(0)
        
            
            
    to_submit=pd.DataFrame(columns=['datetime','signal','close\open'])
    to_submit['datetime']=datetime
    to_submit['signal']=signal
    to_submit['close\open']=trade
#     to_submit['price']=price
    return to_submit
            
            

In [6]:
def metrics(data,submit):
    capital=1000
    current=0
    temp=0
    portfolio_value=[]
    datetime=[]
    quantity=[]
    
    sharpe=[]
    returns=[]
    risk_free_rate=0.05       
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        if(submit.signal[i]==1):
            if(current==0):
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
                temp=capital/(data.close[i])
                quantity.append(temp)
                capital=0
                current=1
            else:
                capital=capital-(temp*data.close[i])
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                quantity.append(temp)
                portfolio_value.append(capital)
                temp=0
                current=0
                
        elif(submit.signal[i]==-1):
            if(current==0):
                portfolio_value.append(capital)
                temp=capital/(data.close[i])
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                capital=capital+(temp*data.close[i])
                
                current=-1
            else:
                capital=capital+(temp*data.close[i])
                quantity.append(temp)
                current=0
                temp=0
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
        
        else:
            if(current==0):
                portfolio_value.append(capital)
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            elif(current==1):
                portfolio_value.append(capital+(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital+(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            else:
                portfolio_value.append(capital-(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital-(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
#     a['sharpe']=sharpe
    a['quantity']=quantity
    
#     a['daily_return'] = a['portfolio value'].pct_change()
#     daily_volatility = a['daily_return'].std()
#     annual_volatility = daily_volatility * math.sqrt(252)
#     risk_free_rate = 0.03
    
#     a['excess_return'] = a['daily_return'] - risk_free_rate / 252
#     cumulative_excess_return = a['excess_return'].sum()
#     negative_returns = [r for r in a['excess_return'] if r < 0]
#     cumulative_volatility = a['daily_return'].std() * math.sqrt(252)
#     hello=np.std(negative_returns)*math.sqrt(252)
#     sharpe_ratio = cumulative_excess_return / cumulative_volatility
#     sortino_ratio=cumulative_excess_return / hello

    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
    a['quantity']=quantity
    a['daily_return'] = a['portfolio value'].pct_change()
    daily_volatility = a['daily_return'].std()
    annual_volatility = daily_volatility * math.sqrt(365)
    annual_volatility_negative = a[a['daily_return'] < 0]['daily_return'].std() * math.sqrt(365)
    risk_free_rate = 0.05
    a['excess_return'] = (a['daily_return'] - risk_free_rate)
    annual_excess_return = (a['daily_return'].mean()*365 - risk_free_rate)
    cumulative_excess_return = a['excess_return'].sum()
    negative_returns = [r for r in a['excess_return'] if r < 0]
    cumulative_volatility = a['daily_return'].std()*math.sqrt(252)
    hello=np.std(negative_returns)*math.sqrt(252)
    sharpe_ratio = annual_excess_return / annual_volatility
    sortino_ratio = annual_excess_return / annual_volatility_negative
    
    
    
    start=[]
    end=[]
    start_value=[]
    end_value=[]
    duration=[]
    P_and_L=[]
    return_for_every_trade=[]
    pointer=0
    drawdown=[]
    for i in range(len(submit.signal)):
        if(submit.signal[i]==1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
#                 drawdown.append(100*((portfolio_value[start[-1]]/minimum)-1))
                
        elif(submit.signal[i]==-1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=-1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
                
    b=pd.DataFrame(columns=['start','end'])
    b['start']=start
    b['end']=end
    b['duration']=duration
    b['return for every trade']=return_for_every_trade
    b['P and L']=P_and_L
    b['drawdown']=drawdown
    
    

    gross_profit=[]
    gross_loss=[]
    for i in range(len(P_and_L)):
        if(P_and_L[i]>0):
            gross_profit.append(P_and_L[i])
        else:
            gross_loss.append(P_and_L[i])
    print("GROSS PROFIT: ",np.sum(gross_profit))
    print("GROSS LOSS: ",np.sum(gross_loss))
    print("NET P and L: ",np.sum(P_and_L))
    print("TOTAL CLOSED TRADES: ",len(start))
    print("WIN RATE: ",100*(len(gross_profit)/(len(gross_profit)+len(gross_loss))))
    print("AVG WINNING TRADE: ",np.mean(gross_profit))
    print("AVG LOSING TRADE: ",np.mean(gross_loss))
    print("Largest Losing trade: ",min(gross_loss))
    print("Largest winning trade: ",max(gross_profit))
    print("Avg holding duration: ",np.mean(duration))
    
   
    
    print("Sharpe Ratio: ",sharpe_ratio)


#     std_dev_negative = np.std(negative_returns)
            
    print("Sortino Ratio: ",sortino_ratio)
    print("RETURNS: ",((np.sum(P_and_L))/10))
    print("MAX Drawdown: (in %) ",np.max(drawdown))
    
    return a,b
                

In [7]:
daily_metrics,only_trade=metrics(data,to_submit(data,output))

C:\Users\Rachit\AppData\Local\Temp\ipykernel_5520\3408008015.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\Rachit\AppData\Local\Temp\ipykernel_5520\3408008015.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  828.5865370284262
GROSS LOSS:  -4924.348070358385
NET P and L:  -4095.761533329958
TOTAL CLOSED TRADES:  7
WIN RATE:  42.857142857142854
AVG WINNING TRADE:  276.19551234280874
AVG LOSING TRADE:  -1231.0870175895961
Largest Losing trade:  -3700.554515065993
Largest winning trade:  557.1282618857431
Avg holding duration:  204.0
Sharpe Ratio:  0.7367479554590185
Sortino Ratio:  1.0601654590309608
RETURNS:  -409.57615333299583
MAX Drawdown: (in %)  315.5161047627009


In [8]:
daily_metrics

,datetime,portfolio value,quantity,daily_return,excess_return
0,2018-01-01,1000.000000,0.000000,NaN,NaN
1,2018-01-02,1000.000000,0.000000,0.000000,-0.050000
2,2018-01-03,1000.000000,0.000000,0.000000,-0.050000
3,2018-01-04,1000.000000,0.000000,0.000000,-0.050000
4,2018-01-05,1000.000000,0.000000,0.000000,-0.050000
...,...,...,...,...,...
1468,2022-01-08,-2951.025704,-0.072543,-0.024643,-0.074643
1469,2022-01-09,-3044.400546,-0.072543,0.031641,-0.018359
1470,2022-01-10,-3018.110331,-0.072543,-0.008636,-0.058636
1471,2022-01-11,-3101.564953,-0.072543,0.027651,-0.022349


In [9]:
only_trade

,start,end,duration,return for every trade,P and L,drawdown
0,13,24,11,15.190351,151.903505,24.750889
1,37,176,139,-25.000243,-287.978677,47.652174
2,180,1100,920,-428.342188,-3700.554515,315.516105
3,1101,1127,26,-4.214677,119.554770,-36.823619
4,1137,1224,87,20.536422,-557.989975,-43.630011
5,1226,1342,116,11.536410,-377.824903,-47.187111
6,1343,1472,129,-15.251713,557.128262,-66.649730


In [10]:

def only_long_then_short(data,output):
    datetime=[]
    signal=[]
    trade=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        trade.append('close')
#         price.append(data.close[i])
        if(i==len(data.close)-1):
            if(current==1):
                signal.append(-1)
                break
            else:
                signal.append(0)
                break
            
        
        if(output.signal[i]==1):
            if(current==0):
                signal.append(1)
                current=1
            else:
                signal.append(0)
        elif(output.signal[i]==-1):
            if(current==1):
                signal.append(-1)
                current=0
            else:
                signal.append(0)
            
        else:
            if(current==0):
                signal.append(0)
            else:
                signal.append(0)
    a=pd.DataFrame(columns=['datetime','signal','trade','price'])
    a['price']=price
    a['datetime']=datetime
    a['signal']=signal
    a['trade']=trade
    return a
            

In [11]:
only_long_then_short_daily,only_long_then_short_trade=metrics(data,only_long_then_short(data,output))

C:\Users\Rachit\AppData\Local\Temp\ipykernel_5520\3408008015.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\Rachit\AppData\Local\Temp\ipykernel_5520\3408008015.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  119.71123450599259
GROSS LOSS:  -548.153658165232
NET P and L:  -428.44242365923935
TOTAL CLOSED TRADES:  4
WIN RATE:  50.0
AVG WINNING TRADE:  59.855617252996296
AVG LOSING TRADE:  -274.076829082616
Largest Losing trade:  -451.6684444444445
Largest winning trade:  113.8179523535996
Avg holding duration:  99.0
Sharpe Ratio:  -0.20773178711276452
Sortino Ratio:  -0.15056136033620382
RETURNS:  -42.84424236592393
MAX Drawdown: (in %)  48.06076550963252


In [12]:
only_long_then_short_daily,

(        datetime  portfolio value  quantity  daily_return  excess_return
 0     2018-01-01      1000.000000  0.000000           NaN            NaN
 1     2018-01-02      1000.000000  0.000000      0.000000      -0.050000
 2     2018-01-03      1000.000000  0.000000      0.000000      -0.050000
 3     2018-01-04      1000.000000  0.000000      0.000000      -0.050000
 4     2018-01-05      1000.000000  0.000000      0.000000      -0.050000
 ...          ...              ...       ...           ...            ...
 1468  2022-01-08       544.835602  0.013393     -0.024643      -0.074643
 1469  2022-01-09       562.075011  0.013393      0.031641      -0.018359
 1470  2022-01-10       557.221158  0.013393     -0.008636      -0.058636
 1471  2022-01-11       572.629038  0.013393      0.027651      -0.022349
 1472  2022-01-12       571.557576  0.013393     -0.001871      -0.051871
 
 [1473 rows x 5 columns],)

In [13]:
only_long_then_short_trade

,start,end,duration,return for every trade,P and L,drawdown
0,24,176,152,-45.166844,-451.668444,48.060766
1,1100,1127,27,1.074766,5.893282,26.913204
2,1137,1224,87,20.536422,113.817952,30.376668
3,1342,1472,130,-14.442969,-96.485214,39.993902
